# Посредник (mediator)

Посредник — это поведенческий паттерн проектирования, который позволяет уменьшить связанность множества классов между собой, благодаря перемещению этих связей в один класс-посредник.


## Пример

Автоматизируем аэродром. Требуется посадить исправный самолет на свободную посадочную полосу в светлое время суток. Классы самолета, посадочной полосы и времени суток, — уже были реализованы другой командой.


In [20]:
from __future__ import annotations
from random import choice
import abc


random_flag = lambda: choice((True, False))


class BaseComponent:
    @property
    def mediator(self) -> AbsMediator:
        return self._mediator

    @mediator.setter
    def mediator(self, mediator: AbsMediator) -> None:
        self._mediator = mediator


class Airplane(BaseComponent):
    def __init__(self, tail_number: str) -> None:
        self.tail_number = tail_number

    def is_serviceable(self) -> bool:
        return random_flag()

    def to_land_the_plane(self) -> None:
        """
        Для проверка возможности посадки идет обращение к посреднику,
        теперь все взаимодействия между объектами будут производится через вызов `notify` У посредника.
        """
        self.mediator.notify(self, "to_land_the_plane")


class LandingStrip(BaseComponent):
    def is_free(self) -> bool:
        return random_flag()


class TimesOfDay(BaseComponent):
    def is_daylight_hours(self) -> bool:
        return random_flag()


class AbsMediator(abc.ABC):
    @abc.abstractmethod
    def notify(self, sender: BaseComponent, event: str) -> None:
        ...


class AirfieldController(AbsMediator):
    def __init__(
        self, component1: Airplane, component2: LandingStrip, component3: TimesOfDay
    ) -> None:
        self._component1 = component1
        self._component1.mediator = self
        self._component2 = component2
        self._component2.mediator = self
        self._component3 = component3
        self._component3.mediator = self

    def notify(self, sender: BaseComponent, event: str) -> None:
        """
        Тут происходит логика проверка возможности посадки самолета.
        Т. е. все взаимодействия между объектами выносятся в отдельный (этот) класс.
        """

        if event != "to_land_the_plane":
            # Условие нужно только для демонстрации того, что метод может обрабатывать разные события
            return

        print("Пытаемся посадить самолет:")

        is_serviceable = self._component1.is_serviceable()
        if not is_serviceable:
            print("[FAIL] Самолет неисправен, слейте топливо перед посадкой")
            return
        print("[ОК] Самолет исправен")

        is_free = self._component2.is_free()
        if not is_free:
            print("[FAIL] Посадочная полоса занята")
            return
        print("[ОК] Посадочная полоса свободна")

        is_daylight_hours = self._component3.is_daylight_hours()
        if not is_daylight_hours:
            print("[FAIL] Плохая видимость, нужно подключить доп. освещение")
            return
        print("[ОК] Видимость отличная")

        print("[SUCCESS] Получено разрешение на посадку")

In [27]:
c1 = Airplane("BT434FAS")
c2 = LandingStrip()
c3 = TimesOfDay()
mediator = AirfieldController(c1, c2, c3)

# Пытаемся посадить самолет
c1.to_land_the_plane()

Пытаемся посадить самолет:
[ОК] Самолет исправен
[ОК] Посадочная полоса свободна
[FAIL] Плохая видимость, нужно подключить доп. освещение
